In [4]:
from typing import List, Tuple
from datetime import datetime
import json
import time
import sqlite3
import emoji
import re
from memory_profiler import memory_usage

file_path = "farmers-protest-tweets-2021-2-4.json"

In [5]:
"""
Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días
{
"date": "2021-02-04",
    "user"={
"id": 123456,
"username": "user1",
}
}

"""


def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    conn = sqlite3.connect("tweets.db")
    c = conn.cursor()

    c.execute(
        """CREATE TABLE IF NOT EXISTS tweets
                 (date TEXT, user TEXT, tweet TEXT)"""
    )
    conn.commit()

    c.execute("SELECT COUNT(*) FROM tweets")
    count = c.fetchone()[0]
    if count == 0:
        try:
            with open(file_path, "r") as file:
                for line in file:
                    tweet = json.loads(line)
                    date = tweet["date"][:10]
                    user = tweet["user"]["username"]
                    tweet_text = tweet["content"]
                    c.execute(
                        "INSERT INTO tweets (date, user, tweet) VALUES (?, ?, ?)",
                        (date, user, tweet_text),
                    )
                    conn.commit()
                
        except FileNotFoundError:
            print(f"El archivo {file_path} no existe.")
            return []
        except Exception as e:
            print(f"Error al leer el archivo: {e}")
            return []

    query_top_dates = """
    SELECT date
    FROM tweets
    GROUP BY date
    ORDER BY COUNT(*) DESC
    LIMIT 10
    """
    c.execute(query_top_dates)
    top_dates = [row[0] for row in c.fetchall()]

    top_users_per_date = []
    for date in top_dates:
        query_top_user = f"""
        SELECT user, COUNT(*) as tweet_count
        FROM tweets
        WHERE date = '{date}'
        GROUP BY user
        ORDER BY tweet_count DESC
        LIMIT 1
        """
        c.execute(query_top_user)
        top_user,_ = c.fetchone()
        top_users_per_date.append(
            (datetime.strptime(date, "%Y-%m-%d").date(), top_user)
        )
    conn.close()

    return top_users_per_date

In [6]:
# BST para almacenar los tweets por fecha y usuario y luego obtener los top 10 usuarios, esto esta pensando para que las proximas consultas sean mas rapidas
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    class BSTNode:
        def __init__(self, date, user):
            self.date = date
            self.users = {user: 1}
            self.left = None
            self.right = None

        def insert(self, date, user):
            if date < self.date:
                if self.left is None:
                    self.left = BSTNode(date, user)
                else:
                    self.left.insert(date, user)
            elif date > self.date:
                if self.right is None:
                    self.right = BSTNode(date, user)
                else:
                    self.right.insert(date, user)
            else:
                if user in self.users:
                    self.users[user] += 1
                else:
                    self.users[user] = 1

        def get_top_users(self):
            data = []
            if self.left:
                data.extend(self.left.get_top_users())
            data.append(
                (
                    self.date,
                    max(self.users, key=self.users.get),
                    sum(self.users.values()),
                )
            )
            if self.right:
                data.extend(self.right.get_top_users())
            return data

    root = None
    try:
        with open(file_path, "r") as file:
            for line in file:
                tweet = json.loads(line)
                date = datetime.fromisoformat(tweet["date"]).date()
                user = tweet["user"]["username"]
                if root is None:
                    root = BSTNode(date, user)
                else:
                    root.insert(date, user)
    except FileNotFoundError:
        print(f"El archivo {file_path} no existe.")
        return []
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
        return []

    all_data = root.get_top_users() if root else []
    top_dates = sorted(all_data, key=lambda x: x[2], reverse=True)[:10]
    return [(date, user) for date, user, _ in top_dates]

In [7]:
"""
Los top 10 emojis más usados con su respectivo conteo
{
"content":"emoji"
}
"""


def q2_time(file_path: str) -> List[Tuple[str, int]]:
    conn = sqlite3.connect("emojis.db")
    c = conn.cursor()

    c.execute(
        """CREATE TABLE IF NOT EXISTS tweets
                 (content TEXT)"""
    )
    conn.commit()

    c.execute("SELECT COUNT(*) FROM tweets")
    count = c.fetchone()[0]
    if count == 0:
        try:
            with open(file_path, "r") as file:
                for line in file:
                    tweet = json.loads(line)
                    tweet_text = tweet["content"]
                    c.execute(
                        "INSERT INTO tweets (content) VALUES (?)",
                        (str(tweet_text),),
                    )
            conn.commit()
        except FileNotFoundError:
            print(f"El archivo {file_path} no existe.")
            return []
        except Exception as e:
            print(f"Error al leer el archivo: {e}")
            return []

    query_top_emojis = """ SELECT content FROM tweets"""
    c.execute(query_top_emojis)

    emoji_counter = {}
    for row in c.fetchall():
        tweet = row[0]
        for char in tweet:
            if char in emoji.UNICODE_EMOJI['en']:
                if char in emoji_counter:
                    emoji_counter[char] += 1
                else:
                    emoji_counter[char] = 1
    conn.close()
        
    return sorted(emoji_counter.items(), key=lambda x: x[1], reverse=True)[:10]

In [8]:
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    class BSTNode:
        def __init__(self, emoji):
            self.emoji = emoji
            self.counter = 1
            self.left = None
            self.right = None

        def insert(self, emoji):
            if emoji < self.emoji:
                if self.left is None:
                    self.left = BSTNode(emoji)
                else:
                    self.left.insert(emoji)
            elif emoji > self.emoji:
                if self.right is None:
                    self.right = BSTNode(emoji)
                else:
                    self.right.insert(emoji)
            else:
                self.counter += 1

        def get_top_emojis(self):
            data = []
            if self.left:
                data.extend(self.left.get_top_emojis())
            data.append((self.emoji, self.counter))
            if self.right:
                data.extend(self.right.get_top_emojis())
            return data

    root = None
    try:
        with open(file_path, "r") as file:
            for line in file:
                tweet = json.loads(line)
                for char in tweet["content"]:
                    if char in emoji.UNICODE_EMOJI["en"]:
                        if root is None:
                            root = BSTNode(char)
                        else:
                            root.insert(char)
                
    except FileNotFoundError:
        print(f"El archivo {file_path} no existe.")
        return []
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
        return []    

    all_data = root.get_top_emojis() if root else []
    return sorted(all_data, key=lambda x: x[1], reverse=True)[:10]

In [9]:
"""
El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos
{
"content":"@user"
}
"""


def q3_time(file_path: str) -> List[Tuple[str, int]]:
    conn = sqlite3.connect("mentions.db")
    c = conn.cursor()

    c.execute(
        """CREATE TABLE IF NOT EXISTS mentions
                 (user TEXT)"""
    )
    conn.commit()

    c.execute("SELECT COUNT(*) FROM mentions")
    count = c.fetchone()[0]
    if count == 0:
        try:
            with open(file_path, "r") as file:
                for line in file:
                    tweet = json.loads(line)
                    tweet_text = tweet["content"]
                    mentions = re.findall(r"@(\w+)", tweet_text)
                    for mention in mentions:
                        c.execute(
                            "INSERT INTO mentions (user) VALUES (?)",
                            (mention,),
                        )
            conn.commit()
        except FileNotFoundError:
            print(f"El archivo {file_path} no existe.")
            return []
        except Exception as e:
            print(f"Error al leer el archivo: {e}")
            return []
        

    query_most_mentioned = """ SELECT user,COUNT(*) as counter FROM mentions GROUP BY user ORDER BY counter DESC LIMIT 10"""
    c.execute(query_most_mentioned)

    most_mentioned = c.fetchall()
    conn.close()
        
    return most_mentioned

In [10]:
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    class BSTNode:
        def __init__(self, user):
            self.user = user
            self.counter = 1
            self.left = None
            self.right = None

        def insert(self, user):
            if user < self.user:
                if self.left is None:
                    self.left = BSTNode(user)
                else:
                    self.left.insert(user)
            elif user > self.user:
                if self.right is None:
                    self.right = BSTNode(user)
                else:
                    self.right.insert(user)
            else:
                self.counter += 1

        def get_most_mentioned(self):
            data = []
            if self.left:
                data.extend(self.left.get_most_mentioned())
            data.append((self.user, self.counter))
            if self.right:
                data.extend(self.right.get_most_mentioned())
            return data

    root = None
    try:
        with open(file_path, "r") as file:
            for line in file:
                tweet = json.loads(line)
                tweet_text = tweet["content"]
                mentions = re.findall(r"@(\w+)", tweet_text)
                for mention in mentions:
                    if root is None:
                        root = BSTNode(mention)
                    else:
                        root.insert(mention)
    except FileNotFoundError:
        print(f"El archivo {file_path} no existe.")
        return []
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
        return []

    all_data = root.get_most_mentioned() if root else []
    return sorted(all_data, key=lambda x: x[1], reverse=True)[:10]

In [25]:
%reload_ext memory_profiler
%memit top_dates = q1_time(file_path)

KeyboardInterrupt: 

In [ ]:
%reload_ext memory_profiler
%memit top_dates = q1_memory(file_path)

peak memory: 87.62 MiB, increment: 4.86 MiB


In [ ]:
%reload_ext memory_profiler
%memit top_dates = q2_time(file_path)

peak memory: 130.04 MiB, increment: 44.05 MiB


In [ ]:
%reload_ext memory_profiler
%memit top_dates = q2_memory(file_path)

peak memory: 89.70 MiB, increment: 0.01 MiB


In [ ]:
%reload_ext memory_profiler
%memit top_dates = q3_time(file_path)

peak memory: 90.70 MiB, increment: 2.13 MiB


In [ ]:
%reload_ext memory_profiler
%memit top_dates = q3_memory(file_path)

peak memory: 89.71 MiB, increment: 0.74 MiB


In [11]:
def time_memory(func, file_path, func_name):
    start_time = time.time()
    result = func(file_path)
    end_time = time.time()

    total_time = end_time - start_time

    print(f"{func_name}:")
    print("Result:", result)
    print(f"Time: {total_time:.4f} s")

    return total_time

In [12]:
print("Q1")
time_memory(q1_time, file_path, "q1_time")
time_memory(q1_memory, file_path, "q1_memory")

print("Q2")
time_memory(q2_time, file_path, "q2_time")
time_memory(q2_memory, file_path, "q2_memory")

print("Q3")
time_memory(q3_time, file_path, "q3_time")
time_memory(q3_memory, file_path, "q3_memory")

Q1
q1_time:
Result: [(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
Time: 278.6596 s
q1_memory:
Result: [(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
Time: 3.1818 s
Q2
q2_time:
R

3.2994537353515625